# CSP(1 point)
In the following code , only two of the function usages are needed. One is `is_valid(self, state)`, which is to determine if the current state is legal; the other is `is_satisfy(self, state)`, which is to determine if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
In the first line of the code you can see N = 5, this is the size of the test.
Then in the test_block(see the last cell), `render` indicates whether to use a graphical interface representation, and then `method` indicates which bts function are used(`bts` or `improving_bts`or`min_conflict`).


## Question 1: You should complete the function `bts()`. (0.8 points)
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. 
## Question 2: You should complete the function `improving_bts()` or `min_conflict()`. (0.2 points)
For `improving_bts()`, You can select one or more methods of the three methods below:Minimum Remaining Value, Least constraining value, Forward checking. You should have a good performance **when N = 16 without GUI**. 

For `min_conflict()`, you should have a good performance **when N = 100 without GUI**.

### DDL: 22:00, Nov.1st
The practice will be checked in this lab class or the next lab class(before **Nov.1st**) by teachers or SAs. 
#### What will be tested: 
* That you understand every line of your code, not just copy from somewhere 
* That your program compiles correctly
* Correctness of the program logic 
* That the result is obtained in a reasonable time 

#### Grading: 
* Submissions in this lab class: 1.1 points.
* Submissions on time: 1 point.
* Late submissions within 2 weeks after the deadline : 0.8 points.


In [1]:
import numpy as np
import time

N = 7 # You should change the test size here !!!

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        """
        check if the current board meets the win condition
        """
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

## BTS: Backtracking search

In [2]:
def bts(problem):
    action_stack = []
    n = problem.n
    black_hole = [[] for _ in range(n)]
    while not problem.is_satisfy(action_stack):
        if len(action_stack) < n:
            for i in range(n):
                if (i not in black_hole[len(action_stack)] and
                        problem.is_valid(action_stack + [(len(action_stack), i)])):
                    action_stack.append((len(action_stack), i))
                    break
                elif i == n - 1:
                    if len(action_stack) == 0:
                        return []
                    black_hole[len(action_stack) - 1].append(action_stack[-1][1])
                    black_hole[len(action_stack)] = []
                    action_stack.pop()
        yield action_stack

## Improving BTS
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [3]:
def improving_bts(problem):
    action_stack = []
    while not problem.is_satisfy(action_stack):
        # TODO: Implement improving bts logic here
        
        yield action_stack

## Local search for CSP(min-conflict algorithm)

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts

    * Reassign the value of this variable to another value that with the least constraint conflicts with other variables

In [4]:
def min_conflict(problem):
    action_stack = []
    while not problem.is_satisfy(action_stack):
        # TODO: Implement min_conflict algorithm logic here
        
        yield action_stack

In [5]:

# test_block
n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
render = False # here to select GUI or not
method = bts  # here to change your method; bts or improving_bts or min_conflict
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in method(p):
        p.print_state(actions)
        pass
    print(time.time() - start_time)


_______________
|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|Q|·|·|
|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|Q|·|·|
|·|Q|·|·|·|·|·|
|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|Q|·|·|
|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
---------------
_______________
|Q|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|
|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|
--------